In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

# load data
file_path = '/content/drive/MyDrive/test_teknikal_kecilin_syalwadea/400exhuma_preprocessed_data.csv'
df = pd.read_csv(file_path, sep=',')

texts = df['full_text'].tolist()
labels = df['sentiment'].tolist()

# tokenize
max_words = 5000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

max_sequence_length = 200
data = pad_sequences(sequences, maxlen=max_sequence_length)

labels = pd.get_dummies(labels)

# split train test
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# LSTM model
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=100, input_length=max_sequence_length))
model.add(LSTM(units=64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(units=len(labels.columns), activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train
model.fit(X_train, y_train, epochs=200, batch_size=32, validation_data=(X_test, y_test))

# Evaluate
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

Epoch 1/200
10/10 [==============================] - 14s 713ms/step - loss: 1.5783 - accuracy: 0.3719 - val_loss: 1.5017 - val_accuracy: 0.5500
Epoch 2/200
10/10 [==============================] - 5s 476ms/step - loss: 1.4023 - accuracy: 0.5375 - val_loss: 1.2850 - val_accuracy: 0.4875
Epoch 3/200
10/10 [==============================] - 6s 589ms/step - loss: 1.2942 - accuracy: 0.4844 - val_loss: 1.1826 - val_accuracy: 0.5500
Epoch 4/200
10/10 [==============================] - 5s 488ms/step - loss: 1.1875 - accuracy: 0.5250 - val_loss: 1.1009 - val_accuracy: 0.5500
Epoch 5/200
10/10 [==============================] - 4s 415ms/step - loss: 1.1003 - accuracy: 0.5594 - val_loss: 1.0943 - val_accuracy: 0.5625
Epoch 6/200
10/10 [==============================] - 5s 561ms/step - loss: 1.0003 - accuracy: 0.6156 - val_loss: 1.1750 - val_accuracy: 0.5125
Epoch 7/200
10/10 [==============================] - 5s 454ms/step - loss: 0.8726 - accuracy: 0.6906 - val_loss: 1.0554 - val_accuracy: 0.587

In [2]:
# Save the model to a file
model.save('lstm_sentiment_model_200.h5')

# Optionally, save the tokenizer as well for later use during inference
import pickle

with open('tokenizer.pkl', 'wb') as tokenizer_file:
    pickle.dump(tokenizer, tokenizer_file)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [9]:
from keras.models import load_model
import pickle

# Load the model
loaded_model = load_model('lstm_sentiment_model_200.h5')

# Load the tokenizer
with open('tokenizer.pkl', 'rb') as tokenizer_file:
    loaded_tokenizer = pickle.load(tokenizer_file)

# New sentence for testing
test_sentence = 'film exhuma itu bagus tapi menakutkan'
# new_sentence = preprocess(test_sentence)
# Tokenize and pad the new sentence
new_sequence = loaded_tokenizer.texts_to_sequences([test_sentence])
new_data = pad_sequences(new_sequence, maxlen=max_sequence_length)

# Predict sentiment for the new sentence
predictions = loaded_model.predict(new_data)

# Get the predicted sentiment label
predicted_sentiment_label = labels.columns[predictions.argmax(axis=1)[0]]
print(f"preprocessing: {test_sentence}")
print(f"new_sequence: {new_sequence}")
print(f"new_data: {new_data}")
print(f"Predicted Sentiment: {predicted_sentiment_label}")

1/1 [==============================] - 0s 260ms/step
preprocessing: film exhuma itu bagus tapi menakutkan
new_sequence: [[4, 1, 5]]
new_data: [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 4 1 5]]
Predicted Sentiment: positive
